In [48]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [49]:
sequence_lenght = 4
batch_size = 1
input_dim = 512
d_model = 512

x  = torch.randn((batch_size, sequence_lenght, input_dim))

In [50]:
print(x)

tensor([[[-1.4203,  1.0772,  2.5285,  ..., -0.4672,  1.4600,  1.6652],
         [ 0.5190,  1.4754,  1.5842,  ...,  0.8647, -0.9960,  0.0946],
         [-0.4422,  1.3927, -1.7449,  ..., -0.0737, -1.4765, -0.1480],
         [-0.5101, -0.8358,  0.5738,  ..., -0.9981,  0.5796, -0.6100]]])


In [51]:
qkv_layer = nn.Linear(input_dim, 3 * d_model) 
qkv = qkv_layer(x)
print(qkv.size())

torch.Size([1, 4, 1536])


In [52]:
num_heads = 8
head_dim = d_model // num_heads

qkv = qkv.reshape(batch_size, sequence_lenght, num_heads, 3 * head_dim)

In [53]:
qkv.shape

torch.Size([1, 4, 8, 192])

In [54]:
qkv =  qkv.permute(0,2,1,3)
qkv.shape

torch.Size([1, 8, 4, 192])

In [55]:
q, k, v = qkv.chunk(3, dim=-1)
q.shape, k.shape, v.shape

(torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]),
 torch.Size([1, 8, 4, 64]))

In [56]:
dk = q.size()[-1]
scaled = torch.matmul(q, k.transpose(-2,-1)) / math.sqrt(dk)
print(scaled.shape)

torch.Size([1, 8, 4, 4])


In [57]:
mask = torch.full(scaled.size(), float('-inf'))
mask = torch.triu(mask, diagonal= 1)
mask[0][0]

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [58]:
(scaled+mask)[0][0]


tensor([[ 0.4360,    -inf,    -inf,    -inf],
        [ 0.6929, -0.2906,    -inf,    -inf],
        [-0.4178, -0.1820, -0.0706,    -inf],
        [-0.0824,  0.0906,  0.1328, -0.0998]], grad_fn=<SelectBackward0>)

In [59]:
scaled += mask

In [60]:
attention = F.softmax(scaled, dim=-1)
attention.shape

torch.Size([1, 8, 4, 4])

In [61]:
values = torch.matmul(attention,v)
values.shape

torch.Size([1, 8, 4, 64])

In [62]:
import math

def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    if mask is not None:
        scaled += mask
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

In [63]:
values, attention = scaled_dot_product(q, k, v, mask=mask)
attention.shape

torch.Size([1, 8, 4, 4])

In [64]:
values.size()


torch.Size([1, 8, 4, 64])

In [65]:
values = values.reshape(batch_size, sequence_lenght, num_heads * head_dim)
values.size()

torch.Size([1, 4, 512])

In [66]:
linear_layer = nn.Linear(d_model, d_model)
out = linear_layer(values)
out.shape

torch.Size([1, 4, 512])

In [67]:
out

tensor([[[-0.0863,  0.3125,  0.4602,  ..., -0.1259, -0.0295,  0.0409],
         [ 0.2172,  0.1198, -0.1014,  ..., -0.5288, -0.3600, -0.1468],
         [-0.0713,  0.2943,  0.2578,  ...,  0.0596, -0.0975,  0.1338],
         [-0.4106,  0.1958, -0.1963,  ..., -0.2894, -0.0833,  0.5440]]],
       grad_fn=<ViewBackward0>)